In [ ]:
!pip install fasttext

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 68 kB 6.8 MB/s 
  Using cached pybind11-2.10.0-py3-none-any.whl (213 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=3164524 sha256=572d089421ca8b81fa91fa0d6dda693dc203f241efbfe6c5acb9a4028f103b72
  Stored in directory: /root/.cache/pip/wheels/4e/ca/bf/b020d2be95f7641801a6597a29c8f4f19e38f9c02a345bab9b
Successfully built fasttext


In [ ]:
!pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 2.4 MB/s 
     |████████████████████████████████| 8.2 MB 19.6 MB/s 
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13723 sha256=c914576ecc0c57ab7c1d204461d7ce38b90f39c9ce82d053a526a82753367919
  Stored in directory: /root/.cache/pip/wheels/72/b0/3f/1d95f96ff986c7dfffe46ce2be4062f38ebd04b506c77c81b9
Successfully built docopt


In [ ]:
!pip install natasha

In [ ]:
import fasttext
import pandas as pd
import pymorphy2
from functools import lru_cache
from multiprocessing import Pool
import numpy as np
import re
from sklearn.model_selection import train_test_split
from tqdm import tqdm_notebook as tqdm

In [ ]:
df = pd.read_excel("/content/1 плох раб.xlsx")
df = df[['Примечание','Подтип нарушения']]
df['Подтип нарушения'] = df['Подтип нарушения'].apply(lambda x: x.replace(" ", "_"))

In [ ]:
dfv = pd.read_excel("/content/Новая разметка 2020-май2022.xlsx")
dfv = dfv[['Причина', 'Примечание','Подтип нарушения']]
dfv['Подтип нарушения'] = dfv['Подтип нарушения'].apply(lambda x: x.replace(" ", "_"))

In [ ]:
df['Подтип нарушения'].value_counts()

In [ ]:
df.head()

,Примечание,Подтип нарушения
0,Не идет на ход. Вагон(Витязь) 2020г.в. (экспл....,Техническая_неисправность_вагона
1,Скопление воды . Сообщил 3-30125 в 11-05. пере...,Скопление_воды/снега/грязи_на_путях
2,В салоне трамвая стало плохо женщине.103 в 10:...,Плохое_самочувствие_пассажира
3,При следовании в депо у вагона 31346 произошел...,Сход_вагона
4,"Сход вагона тягач № 323,3223, аварийная бригад...",Сход_вагона


In [ ]:
# df[['Примечание', 'Причина']].apply(lambda x: split(x))
df['Примечание'] = df["Примечание"].astype(str) + " " + df["Причина"].astype(str).str.zfill(6)

Пайморфи

In [ ]:
m = pymorphy2.MorphAnalyzer()

regex = re.compile("[А-Яа-я=!\)\()\_\%/|]+")

def words_only(text, regex=regex):
    try:
        text = re.sub(r'[АВЕКМНОРСТУХ]\d{3}[АВЕКМНОРСТУХУ33]{2}|[АВЕКМНОРСТУХ]\s\d{3}[АВЕКМНОРСТУХУ33]{2}|[АВЕКМНОРСТУХ]\s\d{3}\s[АВЕКМНОРСТУХУ33]{2}|\s[АВЕКМНОРСТУХ]{2}\s', '', text)
        text = text.replace("сообщил", "").replace("</s>", '')
        text = re.sub(r'.иг..ов|Потапова|г/вып|г.в.|Сообщено|Голованов|на месте|Дижение осстанолено|ыод|Движение восстановлено|\Wв\W', '', text)
        regex = regex.findall(text.replace("(", ' ').replace(")", " "))
        
        return regex
    except:
        return []
def lemmatize(text, pymorphy=m):
    try:
        return " ".join([pymorphy.parse(w)[0].normal_form for w in text])
    except:
        return " " 
# получаем отчищенный и токенезированный текст
def clean_text(text):
    return lemmatize(words_only(text))

with Pool(8) as p:
    lemmas = list(tqdm(p.imap(clean_text, df['Примечание']), total=len(df)))

    
df['lemmas'] = lemmas

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/5778 [00:00<?, ?it/s]

In [ ]:
df['Примечание'][:5].to_list()

In [ ]:
lemmas[:5]

In [ ]:

with Pool(8) as p:
    lemmas = list(tqdm(p.imap(clean_text, dfv['Примечание']), total=len(dfv)))
dfv['lemmas'] = lemmas

NameError: ignored

In [ ]:
text = 'Со слов водителя 39-31136 проезжающая грузовая а/м сместила плитку КРП. По акту СП с поднятием плитки КРП-боковой, после разворота строительной техники на 4,5 и 5,0 см, что не угрожало движению трамваев. Прибыв на место КТПЧ-1 Алферов Г.П. дал команду водителю трамвая 39-31136 продолжить движение. Почему водитель не ехал, она не смогла объяснить. Направлена СП Казаков в 16:05, на месте 16:25. Направлены тягачи 321, 323 в 16:04, отмена. Препятствие устранено. Задержаны нулевые рейсы из депо маршрутов 1,16,47.  Движение восстановленно в 16:29. Требуется просмотр камер видеонаблюдения вагона. Случай требует расследования.'

In [ ]:
regex = re.compile("[А-Яа-я=!\)\()\_\%/|]+")
text = re.sub(r'[АВЕКМНОРСТУХ]\d{3}[АВЕКМНОРСТУХУ33]{2}|[АВЕКМНОРСТУХ]\s\d{3}[АВЕКМНОРСТУХУ33]{2}|[АВЕКМНОРСТУХ]\s\d{3}\s[АВЕКМНОРСТУХУ33]{2}|\s[АВЕКМНОРСТУХ]{2}\s', '', text)
text = text.replace("сообщил", "").replace("</s>", '')
text = re.sub(r'.иг..ов|Потапова|Сообщено|Голованов|на месте|Дижение осстанолено|ыод|Движение восстановлено|\Wв\W', '', text)
regex = regex.findall(text.replace("(", ' ').replace(")", " "))
regex

In [ ]:
df.head()

In [ ]:
lemmas[:100]

Наташа

In [ ]:
X_train, X_test, y_train, y_test = np.array([], 'float64'),np.array([], 'float64'),np.array([], 'float64'),np.array([], 'float64')
for typ in df['Подтип нарушения'].unique():
  dfc = df[df['Подтип нарушения'] == typ]
  X = dfc.lemmas.tolist()
  y = dfc['Подтип нарушения'].tolist()
  X, y = np.array(X), np.array(y)
  X_train_p, X_test_1, y_train_1, y_test_1 = train_test_split(X, y, 
                                                              test_size=0.3, 
                                                              random_state=6)
  X_train = np.append(X_train, X_train_p)
  X_test = np.append(X_test, X_test_1)
  y_train = np.append(y_train, y_train_1)
  y_test = np.append(y_test, y_test_1)  
print('total train examples %s' % len(y_train))
print('total test examples %s' % len(y_test))

total train examples 4028
total test examples 1750


In [ ]:
np.unique(y_train, return_counts=True)

(array(['ДТП_автомобиля_с_пешеходом', 'ДТП_с_автомобилем',
        'ДТП_с_трамваем', 'ДТП_трамвая_с_пешеходом',
        'Действия_пассажиров_без_задержания',
        'Действия_пассажиров_с_задержанием', 'Дорожные_работы',
        'Застряло/неисправно_ТС', 'Мероприятия_УГИБДД_(ГУВД)',
        'Наезд_на_посторонний_предмет_(препятствие)',
        'Неисправность/повреждение_контактной_сети',
        'Неисправность_городского_светофора', 'Неисправность_пантографа',
        'Неисправность_устройств_пути',
        'Несвоевременное_окончание_ночных_работ',
        'Неудовлетворительное_состояние_проезжей_части',
        'Обнаружение_бесхозных_предметов', 'Оставлено_ТС',
        'Ошибочные_действия_персонала',
        'Падение_пассажира_во_время_движения',
        'Падение_пассажира_при_отправлении',
        'Падение_пассажира_при_посадке/выходе,_в_стоящем_вагоне',
        'Падение_пассажира_при_торможении', 'Пешеход_мешает_проезду',
        'Плохое_самочувствие_пассажира', 'Плохое_самочувстви

In [ ]:
with open('data.train.txt', "w+") as outfile:
  for i in range(len(X_train)):
    outfile.write('__label__' + y_train[i] + ' ' +   X_train[i] + '\n')

with open('test.txt', "w+") as outfile:
  for i in range(len(X_test)):
    outfile.write('__label__' + y_test[i] + ' ' +   X_test[i] + '\n')

# with open('val.txt', "w+") as outfile:
#   for i in range(len(X_test)):
#     outfile.write('__label__' + y_v[i] + ' ' +   X_v[i] + '\n')


In [ ]:
classifaer = fasttext.train_supervised('data.train.txt', autotuneValidationFile='test.txt') # epoch=10, lr=1.0,  wordNgrams =2 
result = classifaer.test("test.txt")

print('P@1:', result[1])
print('R@1:', result[2])
print('Number of examples', result[0])

P@1: 0.9182857142857143
R@1: 0.9182857142857143
Number of examples 1750


0.9182857142857143

In [ ]:
classifaer.get_nearest_neighbors("водитель")

[(0.9910902380943298, 'вдодителя'),
 (0.9897773265838623, 'водительшихалеев'),
 (0.9877458214759827, 'водимтель'),
 (0.9866105914115906, 'руководитель'),
 (0.9832138419151306, 'водителдь'),
 (0.9806115031242371, 'воодитель'),
 (0.9762468338012695, 'водительть'),
 (0.9605214595794678, '/водимтель'),
 (0.9542784690856934, 'водителчть'),
 (0.9211976528167725, 'заместитель')]

In [ ]:
classifaer.save_model('optimized_test.model')

In [ ]:
# Загружаем, сохраненную ранее модель
model = fasttext.load_model("optimized_test.model")

# Пример классификации одного заголовка
model.predict("Сломалась а/м Н 430 АР 126 на трамвайных путях. Направлен АМПП Потапова в 23:52. Сообщено УГИБДД Голованов в 23:53, на месте 0:24. А/м убрали при помощи трактора. Движение восстановлено в 0:42."
, k=5)


(('__label__Застряло/неисправно_ТС',
  '__label__ДТП_с_автомобилем',
  '__label__ДТП_автомобиля_с_пешеходом',
  '__label__Помеха_на_пути',
  '__label__Оставлено_ТС'),
 array([0.4068844 , 0.15537851, 0.15344545, 0.0698133 , 0.05358527]))

In [ ]:
model = fasttext.load_model("/content/optimized_3_ошибки.model")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

In [ ]:
rename = {
        'Застряло/неисправно ТС': "На пути оставлено/неисправно ТС",
        'Оставлено ТС': "На пути оставлено/неисправно ТС"
    }
list(rename.values())[0]

'На пути оставлено/неисправно ТС'

In [ ]:
df_train = pd.read_excel('all.xlsx')
model = fasttext.load_model("/content/drive/MyDrive/Data/optimized_3_ошибки.model")
# df_train = df_train[~df_train['Примечание'].isnull()]
df_train['subtype'] = ""
df_train['type'] = ""
df_train['gen_type'] = ""
# df_train['Примечание'] = df_train["Примечание"].astype(str) + " " + df_train["Причина"].astype(str).str.zfill(6)
for i, item in enumerate(df_train['description']):
  item = item.replace("\n"," ")
  df_train['subtype'].iloc[i] = model.predict(item)[0][0].replace("__label__", '').replace("_", ' ')
  # print(df_train['subtype'].values)
  df_train['type'].iloc[i] = types[df_train['subtype'].iloc[i]]
  df_train['gen_type'].iloc[i] = general_types[df_train['type'].iloc[i]]

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [ ]:
df_train[['Подтип нарушения', 'Подтип новый нарушения']] # по 1 столбцу, с кучей правок

,Подтип нарушения,Подтип новый нарушения
0,ДТП с участием трамвая,ДТП с трамваем
1,ДТП с участием трамвая,ДТП с трамваем
2,ДТП с участием трамвая,ДТП с трамваем
3,Плохое самочувствие водителя,Плохое самочувствие работника
4,Трамвай встал под изолятором,Ошибочные действия персонала
5,Неисправность пантографа,Неисправность пантографа
6,Повреждение подвижного состава,ДТП трамвая с пешеходом
7,ДТП с участием трамвая,ДТП с трамваем
8,Неисправность/повреждение контактной сети,Неисправность/повреждение контактной сети
9,Посторонний предмет на контактной сети,Посторонний предмет на контактной сети


In [ ]:
df_train[['Подтип нарушения', 'Подтип новый нарушения']] # иделаьный

,Подтип нарушения,Подтип новый нарушения
0,Оставлено ТС,Оставлено ТС
1,Проверка подвижного состава при выезде из депо,Проверка подвижного состава при выезде из депо
2,Несвоевр. оконч. ночных работ,Несвоевременное окончание ночных работ
3,Проверка подвижного состава при выезде из депо,Проверка подвижного состава при выезде из депо
4,Повреждение подвижного состава,Пропажа напряжения со стороны Мосэнерго
...,...,...
94,NaN,ДТП с трамваем
95,NaN,Падение пассажира при торможении
96,NaN,ДТП с автомобилем
97,NaN,Оставлено ТС


In [ ]:
df_train[['Подтип нарушения', 'Подтип новый нарушения']] # по 1 столбцу, с кучей правок

,Подтип нарушения,Подтип новый нарушения
0,Оставлено ТС,Оставлено ТС
1,Проверка подвижного состава при выезде из депо,Проверка подвижного состава при выезде из депо
2,Несвоевр. оконч. ночных работ,Несвоевременное окончание ночных работ
3,Проверка подвижного состава при выезде из депо,Проверка подвижного состава при выезде из депо
4,Повреждение подвижного состава,Пропажа напряжения со стороны Мосэнерго
...,...,...
94,NaN,ДТП с трамваем
95,NaN,Падение пассажира при торможении
96,NaN,ДТП с автомобилем
97,NaN,Оставлено ТС


In [ ]:
df_train.to_excel('dsd.xlsx')

In [ ]:

types = {
    'ДТП с трамваем': 'ДТП с участием трамвая',
    'ДТП трамвая с пешеходом': 'ДТП с участием трамвая',
    'ДТП с автомобилем': 'ДТП без участия трамвая',
    'ДТП автомобиля с пешеходом': 'ДТП без участия трамвая',
    'Техническая неисправность вагона': 'Техническая неисправность трамвая',
    'Неисправность пантографа': 'Техническая неисправность трамвая',
    'Неисправность устройств пути': 'Неисправность/повреждение инфраструктуры',
    'Повреждение кабеля': 'Неисправность/повреждение инфраструктуры',
    'Посторонний предмет на контактной сети': 'Неисправность/повреждение инфраструктуры',
    'Неисправность/повреждение контактной сети': 'Неисправность/повреждение инфраструктуры',
    'Повреждение контактной сети пантографом': 'Неисправность/повреждение инфраструктуры',
    'Разъезд на стрелке': 'Неисправность/повреждение инфраструктуры',
    'Сход вагона': 'Сход вагона',
    'Обнаружение бесхозных предметов': 'Нарушения, связанные с действиями пассажиров',
    'Плохое самочувствие пассажира': 'Нарушения, связанные с действиями пассажиров',
    'Действия пассажиров с задержанием': 'Нарушения, связанные с действиями пассажиров',
    'Действия пассажиров без задержания': 'Нарушения, связанные с действиями пассажиров',
    'Пешеход мешает проезду': 'Нарушения, связанные с действиями пассажиров',
    'Падение пассажира при посадке/выходе, в стоящем вагоне': 'Нарушения, связанные с действиями пассажиров',
    'Падение пассажира во время движения, в стоящем вагоне': 'Нарушения, связанные с действиями пассажиров',
    'Падение пассажира при отправлении': 'Нарушения, связанные с действиями пассажиров',
    'Падение пассажира при торможении': 'Нарушения, связанные с действиями пассажиров',
    'Плохое самочувствие работника': 'Нарушения, связанные с работниками',
    'Работник получил травму': 'Нарушения, связанные с работниками',
    'Несвоевременное окончание ночных работ': 'Препятствия движению',
    'Дорожные работы': 'Препятствия движению',
    'Неудовлетворительное состояние проезжей части': 'Препятствия движению',
    'Помеха на пути': 'Препятствия движению',
    'Скопление постороннего транспорта': 'Препятствия движению',
    'Скопление воды/снега/грязи на путях': 'Препятствия движению',
    'Застряло/неисправно ТС': 'Препятствия движению',
    'Оставлено ТС': 'Препятствия движению',
    'Наезд на посторонний предмет (препятствие)': 'Препятствия движению',
    'Скопление вагонов при выезде/въезде из депо': 'Препятствия движению',
    'Неисправность городского светофора': 'Неисправность/повреждение городской инфраструктуры',
    'Пропажа напряжения со стороны Мосэнерго': 'Неисправность/повреждение городской инфраструктуры',
    'Проверка подвижного состава при выезде из депо': 'Прочие происшествия',
    'ЧП на территории города': 'Прочие происшествия',
    'Мероприятия УГИБДД (ГУВД)': 'Прочие происшествия',
    'Трамвай встал под изолятором': 'Прочие происшествия',
    'Повреждение подвижного состава': 'Прочие происшествия',
    'Ошибочные действия персонала': 'Ошибочные действия персонала'
}
general_types = {
    'ДТП с участием трамвая': 'ДТП',
    'ДТП без участия трамвая': 'ДТП',
    'Техническая неисправность трамвая': 'Неисправность инфраструктуры/подвижного состава',
    'Неисправность/повреждение инфраструктуры': 'Неисправность инфраструктуры/подвижного состава',
    'Сход вагона': 'Неисправность инфраструктуры/подвижного состава',
    'Нарушения, связанные с действиями пассажиров': 'Прочие',
    'Нарушения, связанные с работниками': 'Прочие',
    'Прочие происшествия': 'Прочие',
    'Неисправность/повреждение городской инфраструктуры': 'Прочие',
    'Препятствия движению': 'Прочие',
    'Ошибочные действия персонала': 'Ошибочные действия персонала'
}


In [ ]:
df_x = pd.read_excel('/content/new (1).xlsx')

In [ ]:
df_x = df_x[['Примечание','Подтип нарушения']]
df_x['Подтип нарушения'] = df_x['Подтип нарушения'].apply(lambda x: x.replace(" ", "_"))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
X = df_x.lemmas.tolist()
y = df_x['Подтип нарушения'].tolist()

X, y = np.array(X), np.array(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [ ]:
with open('data.train.txt', "w+") as outfile:
  for i in range(len(X_train)):
    outfile.write('__label__' + y_train[i] + ' ' +   X_train[i] + '\n')

with open('test.txt', "w+") as outfile:
  for i in range(len(X_test)):
    outfile.write('__label__' + y_test[i] + ' ' +   X_test[i] + '\n')


In [ ]:
fname = 'optimized.model'

In [ ]:
import fasttext
# from gensim.models.fasttext import load_facebook_model

In [ ]:
gensim.__version__

'4.2.0'

In [ ]:
! pip3 install --upgrade gensim

In [ ]:
!pip install fasttext

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 68 kB 4.9 MB/s 
  Using cached pybind11-2.10.1-py3-none-any.whl (216 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=3158267 sha256=32361134ad2ecd8eb00d2716270b51341621c582be59dbdbd53464034ccbcb87
  Stored in directory: /root/.cache/pip/wheels/4e/ca/bf/b020d2be95f7641801a6597a29c8f4f19e38f9c02a345bab9b
Successfully built fasttext


In [ ]:
import fasttext

In [ ]:
model = fasttext.load_model("optimized.model")

print(model)
# model.build_vocab('data.train.txt', update=True)
# model.train('data.train.txt', total_examples=model.corpus_count, epochs=5)

In [ ]:
model.get_analogies("а/м", "машина", '')

[(0.7926570177078247, 'максимально'),
 (0.7660893201828003, 'данилычев'),
 (0.7657509446144104, 'спилить'),
 (0.7522227168083191, 'оэк'),
 (0.7428737282752991, 'роща'),
 (0.7428735494613647, 'ваврентович'),
 (0.7428735494613647, 'воронов'),
 (0.7426000237464905, 'экскаватор'),
 (0.7426000237464905, 'компенсанциооной'),
 (0.742043137550354, 'демонтировать')]

In [ ]:
from natasha import Doc, MorphVocab, Segmenter, NewsEmbedding, NewsMorphTagger
regex = re.compile('[А-Яа-я:!\)\()A-z\_\%/|/]+')
def words_only(text, regex=regex):
  regex = " ".join([w for w in regex.findall(text.replace('(', '').replace(')', ''))])
  return regex

segmenter = Segmenter()
morph_vocab = MorphVocab()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)

def natasha_lemmatize(text):
  try:
    doc = Doc(text)
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)
    for token in doc.tokens:
      # print(len(token.text))
      # if len(token.text) > 3 or token.text in ['в', "до", 'овд', 'c']:
      token.lemmatize(morph_vocab)
    # return {_.text: _.lemma for _ in doc.tokens}
    return " ".join([p.lemma for p in doc.tokens])
  except:
    return " "

# получаем отчищенный и токенезированный текст
def clean_text(text):   
  return natasha_lemmatize(words_only(text))

with Pool(8) as p:
  lemmas = list(tqdm(p.imap(clean_text, df['Примечание']), total=len(df)))

df['lemmas'] = lemmas

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/11290 [00:00<?, ?it/s]

In [ ]:
from imblearn.over_sampling import BorderlineSMOTE, SMOTE, ADASYN, SMOTENC, RandomOverSampler
from imblearn.under_sampling import (RandomUnderSampler, 
                                    NearMiss, 
                                    InstanceHardnessThreshold,
                                    CondensedNearestNeighbour,
                                    EditedNearestNeighbours,
                                    RepeatedEditedNearestNeighbours,
                                    AllKNN,
                                    NeighbourhoodCleaningRule,
                                    OneSidedSelection,
                                    TomekLinks)
from imblearn.combine import SMOTEENN, SMOTETomek
from imblearn.pipeline import make_pipeline

In [ ]:
def train_model(classifier, feature_vector_train, label, feature_vector_valid):
    classifier.fit(feature_vector_train, label)

    predictions = classifier.predict(feature_vector_valid)    

    return metrics.f1_score(valid_y,predictions, average='weighted')

In [ ]:
from sklearn import model_selection, preprocessing, metrics, linear_model, svm

train_x, valid_x, train_y, valid_y = model_selection.train_test_split(df['Примечание'], df['Подтип нарушения'])
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=100000)
tfidf_vect.fit(df['Примечание'])
xtrain_tfidf =  tfidf_vect.transform(train_x)
xvalid_tfidf =  tfidf_vect.transform(valid_x)

In [ ]:
rus = RandomUnderSampler(random_state=0, replacement=True)
rus_xtrain_tfidf, rus_train_y = rus.fit_resample(xtrain_tfidf, train_y)
accuracyrus = train_model(linear_model.LogisticRegression(random_state=0, solver='newton-cg',multi_class='multinomial'),rus_xtrain_tfidf, rus_train_y, xvalid_tfidf)
print ("Logistic regressio RUS, WordLevel TFIDF: ", accuracyrus)

Logistic regressio RUS, WordLevel TFIDF:  0.2291666706363441


In [ ]:
enn_xtrain_tfidf, enn_train_y = EditedNearestNeighbours().fit_resample(xtrain_tfidf, train_y)
accuracy = train_model(linear_model.LogisticRegression(random_state=0, solver='lbfgs',multi_class='multinomial'),enn_xtrain_tfidf, enn_train_y, xvalid_tfidf)
print ("Logistic regression {0}, WordLevel TFIDF: ", accuracy)

Logistic regression {0}, WordLevel TFIDF:  0.44240194421789375


In [ ]:
#Random Over Sampling
ros = RandomOverSampler(random_state=777)
ros_xtrain_tfidf, ros_train_y = ros.fit_resample(xtrain_tfidf, train_y)
accuracyROS = train_model(linear_model.LogisticRegression(random_state=0, solver='lbfgs',multi_class='multinomial'),ros_xtrain_tfidf, ros_train_y, xvalid_tfidf)
print ("Logistic regression ROS, WordLevel TFIDF: ", accuracyROS)

Logistic regression ROS, WordLevel TFIDF:  0.5896376535297474


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [ ]:
sm = SMOTE(random_state=777)
sm_x_train_tfidf, sm_train_y = sm.fit_resample(xtrain_tfidf, train_y)
accuracySMOTE = train_model(linear_model.LogisticRegression(random_state=0, solver='newton-cg',multi_class='multinomial'),sm_xtrain_tfidf, sm_train_y, xvalid_tfidf)
print ("Logistic regression SMOTE, WordLevel TFIDF: ", accuracySMOTE)
#https://stackoverflow.com/questions/49395939/smote-initialisation-expects-n-neighbors-n-samples-but-n-samples-n-neighbo

ValueError: ignored

In [ ]:
ad = ADASYN(random_state=777)
ad_xtrain_tfidf, ad_train_y = ad.fit_resample(xtrain_tfidf, train_y)
accuracyADASYN = train_model(linear_model.LogisticRegression(random_state=0, solver='lbfgs',multi_class='multinomial'),ad_xtrain_tfidf, ad_train_y, xvalid_tfidf)
print ("Logistic regression ADASYN, WordLevel TFIDF: ", accuracyADASYN)

ValueError: ignored

In [ ]:
se = SMOTEENN(random_state=42)
se_xtrain_tfidf, se_train_y = se.fit_resample(xtrain_tfidf, train_y)
accuracy = train_model(linear_model.LogisticRegression(random_state=0, solver='lbfgs',multi_class='multinomial'),se_xtrain_tfidf, se_train_y, xvalid_tfidf)
print ("Logistic regression SMOTEENN: ", accuracy)

ValueError: ignored

In [ ]:
from nlpaug.util.file.download import DownloadUtil


DownloadUtil.download_glove(model_name='glove.6B', dest_dir='.')

In [ ]:
df = pd.read_excel("/content/trams_curr_nrm (12).xlsx")

In [ ]:
df.columns

Index(['№', 'Дата', 'Место задержки', 'Время',
       'Длительность задержки движения', 'Причина', 'Тип нарушения',
       'Подтип нарушения', '№ марш.', '№ маш.', 'Модель трамвая', 'Э/П',
       'Депо', 'Задерж. марш.', 'Принятые меры', 'Исполнители', 'Напр. СП',
       'Приб. СП', 'Напр. СЭХ', 'Приб. СЭХ', 'Напр. тяг.', 'Приб. тяг.',
       'Примечание', 'Место задержки улица', 'Долгота', 'Широта',
       'Причина возникновения общая', 'Вызов ГИБДД', 'Время следования СП',
       'Время следования СЭХ', 'Время следования тяг.',
       'Время до вызова ГИБДД', 'Время оформления'],
      dtype='object')

In [ ]:
list_df = df[['Место задержки', 'Место задержки улица']]

In [ ]:
list_df['Место задержки'].to_list()

In [ ]:
print([x for x in list_df['Место задержки'] for b in x if b in X or x])

TypeError: ignored

In [ ]:
# regex = re.compile("[А-Яа-я0-9X]+")
res = []
regex = re.compile(".х\s|.Х\s")
l = [" ".join(regex.findall(x)) for x in list_df['Место задержки'].to_list()]
for i, f in enumerate(l):
  if f:
    res.append(list_df.iloc[i])

'ТИМИРЯЗЕВСКАЯ УЛ.х Красностуденческий пр-д'

In [ ]:
pd.DataFrame(res).to_excel('street.xlsx')